<a href="https://colab.research.google.com/github/royn5618/Talks_Resources/blob/main/EuroPython2021/Building_Word2Vec_Model_on_Wikipedia_Texts_on_Programming_Languages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this example, I have used SPARQL Query to query the **Wikipedia Page Names**. The names are in **rdfs:label** property. Once I had the list of wikipedia page names, the next step is to get the contents using **Wikipedia Python Library**, process it and create a **Word2Vec model using Gensim**.

In [1]:
!pip install beautifulsoup4
!pip install wikipedia
!pip install SPARQLWrapper
!pip install gensim

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=b00e5c202bf52219426f3ded8c7357d7af1180a7dede364a037498e23460b07d
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia
     |████████████████████████████████| 376 kB 5.5 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 


In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON, N3
import pandas as pd
from bs4 import BeautifulSoup
import wikipedia

import re

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

from gensim.models import Word2Vec # This is Gensim 3.6.0

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
sparql = SPARQLWrapper('https://dbpedia.org/sparql')

In [4]:
def exec_query(sparql, query):
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results = pd.json_normalize(results['results']['bindings'])
    return results

**YAGO** (Yet Another Great Ontology) is an open source knowledge base developed at the Max Planck Institute for Computer Science in Saarbrücken.

Ref - https://en.wikipedia.org/wiki/YAGO_(database)

In [5]:
query = ''' SELECT ?thing, ?label 
WHERE { ?thing a dbo:ProgrammingLanguage; a dbo:Language; a yago:WikicatProgrammingLanguages; rdfs:label ?label .
FILTER(LANG(?label) = "en")} '''
query

' SELECT ?thing, ?label \nWHERE { ?thing a dbo:ProgrammingLanguage; a dbo:Language; a yago:WikicatProgrammingLanguages; rdfs:label ?label .\nFILTER(LANG(?label) = "en")} '

In [6]:
results = exec_query(sparql, query)
results.head()

,thing.type,thing.value,label.type,label.xml:lang,label.value
0,uri,http://dbpedia.org/resource/C++,literal,en,C++
1,uri,http://dbpedia.org/resource/CLU_(programming_l...,literal,en,CLU (programming language)
2,uri,http://dbpedia.org/resource/COBOL,literal,en,COBOL
3,uri,http://dbpedia.org/resource/COMIT,literal,en,COMIT
4,uri,http://dbpedia.org/resource/CPL_(programming_l...,literal,en,CPL (programming language)


In [7]:
wiki_page_list = list(set(results['label.value']))
len(wiki_page_list)

257

In [8]:
wiki_page_list

['IBM RPG',
 'Visual Basic for Applications',
 'Locomotive BASIC',
 'Caml',
 'C--',
 'Ada (programming language)',
 'Opa (programming language)',
 'PLEX (programming language)',
 'Seed7',
 'Clascal',
 'SPARK (programming language)',
 'Not Quite C',
 'Common Intermediate Language',
 'OpenGL Shading Language',
 'Pacbase',
 'Logtalk',
 'ML (programming language)',
 'Common Lisp',
 'Charm++',
 'BCPL',
 'Modula-2',
 'Simula',
 'MAD (programming language)',
 'SR (programming language)',
 'Tea (programming language)',
 'AspectJ',
 'Perl',
 'ALGOL 60',
 'Newsqueak',
 'Arc (programming language)',
 'Fortran',
 'COBOL',
 'Compiled language',
 'Real-time Programming Language',
 'Turing (programming language)',
 'AutoLISP',
 'FX-87',
 'Scientific Vector Language',
 'ALGOL W',
 'OptimJ',
 'OCaml',
 'Verilog',
 'ABSET',
 'Language Of Temporal Ordering Specification',
 'QML',
 'Pharo',
 'APL (programming language)',
 'MPD (programming language)',
 'VHDL',
 'Rebol',
 'Clarion (programming language)',


In [ ]:
%%time
list_wiki_page_objs = []
for each_page in wiki_page_list:
  try:
    list_wiki_page_objs.append(wikipedia.page(each_page).content)
  except Exception as e:
    continue

In [ ]:
len(list_wiki_page_objs)

In [ ]:
list_wiki_page_objs[0]

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

def text_preprocessing(text):
  cleaned_text = re.sub(r'[^a-zA-Z\d\s\']+', '', text)
  tokenized_text = word_tokenize(cleaned_text.lower())
  lemmatized_text = [wordnet_lemmatizer.lemmatize(token) for token in tokenized_text]
  return lemmatized_text

In [ ]:
%%time
list_wiki_page_objs_tokenized = [text_preprocessing(each_wiki_page_content) for each_wiki_page_content in list_wiki_page_objs]

In [ ]:
model = Word2Vec(list_wiki_page_objs_tokenized, max_vocab_size=200, min_count=5)

In [ ]:
model.wv.most_similar('programming')

In [ ]:
pca = PCA(n_components=2)

In [ ]:
vocab = list(model.wv.vocab)

In [ ]:
X_pca = pca.fit_transform(model.wv[vocab])

In [ ]:
plt.figure(figsize=[20, 15])
plt.scatter(X_pca[:, 0], X_pca[:, 1])
for i, word in enumerate(vocab):
    plt.annotate(word, xy = (X_pca[i, 0], X_pca[i, 1]))
plt.xticks([])
plt.yticks([])
plt.show()

Thanks for visiting!